# Задание:
Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

## Imports

In [6]:
!pip install pyspark

In [15]:
import os
import sys
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql import SparkSession

# Настройка переменных окружения для корректной работы PySpark
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Добавление пакета для парсинга XML
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.17.0 pyspark-shell'

In [16]:
# Создание SparkSession
spark = SparkSession.builder \
    .appName("Top Programming Languages 2010-2020") \
    .getOrCreate()

spark

In [5]:
# Загрузка файла posts_sample.xml
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml

# Загрузка файла programming-languages.csv
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv

--2025-03-16 16:23:48--  https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74162295 (71M) [text/plain]
Saving to: ‘posts_sample.xml’

posts_sample.xml    100%[===================>]  70.73M   362KB/s    in 2m 52s  

2025-03-16 16:26:42 (421 KB/s) - ‘posts_sample.xml’ saved [74162295/74162295]



## Solution

### Чтение и анализ данных posts_sample.xml

In [17]:
# Чтение XML-файла с постами
posts_data = spark.read \
    .format('xml') \
    .option('rowTag', 'row') \
    .option("timestampFormat", 'y/M/d H:m:s') \
    .load('posts_sample.xml')

# Вывод информации о данных
print("Схема данных:")
posts_data.printSchema()

print("\nПервые 5 записей:")
posts_data.show(n=5)

print("\nСтатистика по данным:")
posts_data.describe().show()

print("\nОбщее количество записей:")
print(posts_data.count())

Схема данных:
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)


Первые 5 записей:
+-----------------+------------+------------

### Фильтрация постов по дате:

In [18]:
# Определение временного диапазона (2010-2020)
dates = ("2010-01-01", "2020-12-31")

# Фильтрация постов по дате создания
posts_by_date = posts_data.filter(
    F.col("_CreationDate").between(*dates)
)

# Просмотр первых 10 записей
posts_by_date.show(10)

+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|_Tags|_Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|             NULL|        NULL|<p>No. (And more ...|       NULL

### Чтение и анализ данных programming-languages.csv

In [19]:
# Чтение CSV-файла с языками программирования
languages_data = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option("inferSchema", True) \
    .load('programming-languages.csv') \
    .dropna()

# Вывод информации о данных
print("Схема данных:")
languages_data.printSchema()

print("\nПервые 5 записей:")
languages_data.show(n=5)

print("\nСтатистика по данным:")
languages_data.describe().show()

print("\nОбщее количество записей:")
print(languages_data.count())

Схема данных:
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)


Первые 5 записей:
+----------+--------------------+
|      name|       wikipedia_url|
+----------+--------------------+
|   A# .NET|https://en.wikipe...|
|A# (Axiom)|https://en.wikipe...|
|A-0 System|https://en.wikipe...|
|        A+|https://en.wikipe...|
|       A++|https://en.wikipe...|
+----------+--------------------+
only showing top 5 rows


Статистика по данным:
+-------+--------+--------------------+
|summary|    name|       wikipedia_url|
+-------+--------+--------------------+
|  count|     699|                 699|
|   mean|    NULL|                NULL|
| stddev|    NULL|                NULL|
|    min|@Formula|https://en.wikipe...|
|    max|xHarbour|https://en.wikipe...|
+-------+--------+--------------------+


Общее количество записей:
699


In [20]:
# Получение списка названий языков программирования
language_names = [str(row[0]) for row in languages_data.collect()]

### Определение функции для поиска языков в тегах

In [21]:
def includes_name(post):
    """
    Функция определяет, какой язык программирования упомянут в тегах поста.
    Возвращает кортеж (дата создания, название языка или 'No', если язык не найден).
    """
    tag = None
    for name in language_names:
        tag_pattern = f"<{name.lower()}>"
        if tag_pattern in str(post._Tags).lower():
            tag = name
            break
    if tag is None:
        tag = 'No'
    return (post._CreationDate, tag)

## Обработка данных и создание отчета

In [24]:
# Преобразование DataFrame в RDD, применение функции и фильтрация
posts_by_date_rdd = posts_by_date.rdd \
    .map(includes_name) \
    .filter(lambda x: x[1] != 'No')

# Группировка по году и языку, подсчет и сортировка
posts_by_date_rdd_group = posts_by_date_rdd \
    .keyBy(lambda row: (row[0].year, row[1])) \
    .aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2) \
    .sortBy(lambda x: x[1], ascending=False) \
    .collect()

# Список годов (2010-2020) в обратном порядке
years_list = list(range(2010, 2021))[::-1]

# Формирование топ-10 языков для каждого года
df_by_years = []
for year in years_list:
    df_by_years.extend([row for row in posts_by_date_rdd_group if row[0][0] == year][:10])

# Определение структуры строки для DataFrame
row_template = Row('Year', 'Language', 'Count')

# Создание итогового DataFrame
result_df = spark.createDataFrame(
    [row_template(*x, y) for x, y in df_by_years]
)

# Вывод результата
result_df.show()

# Сохранение в формате Parquet
result_df.write.parquet("top_10_languages_between_2010_and_2020.parquet")

+----+----------+-----+
|Year|  Language|Count|
+----+----------+-----+
|2019|    Python|  162|
|2019|JavaScript|  131|
|2019|      Java|   95|
|2019|       PHP|   59|
|2019|         R|   36|
|2019|         C|   14|
|2019|      Dart|    9|
|2019|    MATLAB|    9|
|2019|        Go|    9|
|2019|      Bash|    8|
|2018|    Python|  214|
|2018|JavaScript|  196|
|2018|      Java|  145|
|2018|       PHP|   99|
|2018|         R|   63|
|2018|         C|   24|
|2018|     Scala|   22|
|2018|TypeScript|   21|
|2018|PowerShell|   13|
|2018|      Bash|   12|
+----+----------+-----+
only showing top 20 rows

